# Автоматическая предобработка отчетов отеля

### Цель проекта:
Автоматизировать ручную предобработку отчетов Property Management System (вариант ERP с усеченной функцией CRM системы для отелей) для последующего использования:
- в BI системе
- в центре конверсий Яндекс.Директа
- дальнейшей аналитике клиентской базы
- интеграции CRM-системы (в перспективе)

### Проблематика проекта:

Аналог ERP системы отеля – PMS Fidelio – объединяет в себе очень большой функционал, от работы с кассами и фискальниками и программирования ключей до распределения задач горничным и элементы CRM системы. Политика российского дистрибьютора ПО - максимальная закрытость продукта: не охотно делятся ключами API, а даже если их прогнуть, не предоставляют API документацию. Из-за чего интеграция с современными системами: телефонией, CRM, системами сквозной аналитики и BI практически невозможна.

**Задача проекта:** на базе 2 отчетов, выгружаемых из PMS Fidelio сформировать датасет для дальнейшего использования в маркетинге: BI-дашборда по клиентам, подгрузке к центру конверсий Яндекс.директа и удобного дальнейшего анализа.

Ранее датасет формирровался вручную в экселе, долго причесывался, в итоге полная аналитика по клиентам делалась раз в год из-за высоких трудозатрат. При смене методики расчета некоторых показателей, весь датасет необходимо было обрабатывать с нуля.

**Основные проблемы источника данных:**
- В кастомном отчете по клиентам-броням неверно расчитывается стоимость заказа, поэтому нужно заново подтягивать данные по броням
- Отчет по броням "битый", при выгрузке слетают названия столбцов, есть пустые колонки, много пустых и служебных строк
- Многие данные о клиентах заполняются вручную, нужно предобработать телефонные номера, как ключевой идентификатор и привести в порядок географию, т.к. на базе неё строятся ключевые метрики.
- Необходимо обогатить данные о регионе по кодам номера телефона для улучшения полноты данных.
- Сегментация клиентов внутри системы (индивидуальные гости, корпоративные клиенты, спортивные события) не всегда корректна, нужно заново сегментировать клиентов по формальным признакам.
- Отчет по броням не выгружает полную информацию по отмененным броням
- Данные о дате и времени создания брони хранятся примерно 2,5 года. Для более старых броней их нет, но есть для отмены более старых броней ??
- Нужно сохранить структуру отчета аналогичной ручной предобработке в excel. Поэтому названия столбцов оставим на русском.

### Результат от внедрения
- Кратно сократились трудозатраты на ручную обработку данных: предобработка годового отчета вместо 2 недель занимает около 2 часов с учётом выгрузки данных из PMS Fidelio
- Сценарий предобработки адаптирован на 4 отеля
- Разработан дашборд на Tableau для визуализации портрета клиента и основных финансовых показателей в динамике
- Отчет по клиентам из ежегодного стал ежемесячным в формате дашборда
- Более полные данные о регионе: для отдельных выгрузок количество броней без присвоенного региона уменьшилось с 14% до 6%
- Закреплена методика сегментации гостей

## Предобработка отчетов

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from numpy import median


pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

custom = pd.read_excel ('gg_custom.xlsx')
reserv = pd.read_excel ('gg_11103.xlsx')

hotel = 'GG' #Код отеля, задаётся на каждую тетрадку, константа. Будет применяться для создания уникального ID 
#и как флаг для отедельных этапов предобработки, которые меняются от отеля к отелю.

colors = sns.color_palette('deep')
sns.set_theme(style="whitegrid")

### Формируем датасет

#### Исправляем ошибки выгрузки 

Данные о бронях выгружаются криво, наименования столбцов сбивается, приводим данные в порядок:
- задаем столбцам рабочие названия
- удаляем пустые столбцы
- удаляем пустые строки по ID брони. Если в строке не заполнены данные по ID брони - то это мусорная строка.

In [2]:
reserv = reserv.rename(columns={'Блок':'Гость','Источник':'Комн.№','Группа':'Комнат','Unnamed: 3':'Тип комнаты','Unnamed: 4':'Тариф','Unnamed: 6':'Компания','Unnamed: 8':'Код тарифа','Unnamed: 10':'Агент','Unnamed: 11':'Заезд','Unnamed: 12':'Выезд','Unnamed: 13':'Взрослых','Unnamed: 14':'Детей','Unnamed: 15':'Коды рынка','Unnamed: 17':'Менеджер','Unnamed: 19':'Res ID','Unnamed: 22':'Источник','Unnamed: 24':'Группа'})

In [3]:
reserv = reserv.drop(labels=['Unnamed: 5','Unnamed: 7','Unnamed: 9','Unnamed: 16','Unnamed: 18','Unnamed: 20','Unnamed: 21','Unnamed: 23'], axis=1)
reserv = reserv.dropna(subset='Res ID')

С кастомным отчетом всё ОК, но в нём неверно считается доход, поэтому будем его пересчитывать по данным из отчёта о бронях

#### Объединяем 2 отчёта

In [4]:
data = custom.merge(reserv, how='outer', on='Res ID')

#### Предобработка датасета

**Приводим столбцы порядок:**
- Даты к формату даты
- Удаляем дубликаты стобцов из 2 отчетов, полученные при слиянии
- Удаляем столбцы, которые для нас неинформативны
- Восстанавливаем порядок столбцов, утвержденный в ранних вариантах ручной обработки
- Делаем удобные наименования столбцов для дальнейшей отчетности

In [5]:
data['Дата созд'] = pd.to_datetime(data['Дата созд'],errors='coerce', dayfirst=True)
data.insert(0, 'Отель', hotel)

In [6]:
data = data.drop(labels=['No E-mailing','Гость','Код тарифа','Заезд_y','Выезд_y','Взрослых','Детей','Источник','Группа_y','Сумма'], axis=1)

In [7]:
data['Res ID'] = data['Res ID'].astype(int)
data['Д. рожд'] = pd.to_datetime(data['Д. рожд'],errors='coerce', dayfirst=True)
data['Дата соз. проф.'] = pd.to_datetime(data['Дата соз. проф.'],errors='coerce', dayfirst=True)

#### Вычисляем необходимые значения
- Продолжительность пребывания
- Окно бронирования (от даты бронирования до даты заезда)
- Доход с брони (цена за сутки Х количество суток)
- Возраст гостя (от даты рождения до даты заезда)

In [8]:
data.insert(0, 'ID', (data['Отель'] + data['Res ID'].astype(str)))
data.insert(4, 'Окно бронир', (data['Заезд_x'] - data['Дата созд']).dt.days+1)
data = data.rename(columns={'Тариф_x':'Код тарифа','Заезд_x':'Заезд','Выезд_x':'Выезд','ИСТ':'Код источника','Имя':'ФИО','Дата соз. проф.':'Дата созд. проф.','Ночей':'Накопл. ночей','Н\ноч':'Накопл. заездов','Группа_x':'Группа'})
data.insert(6, 'Категория номера', data['Тип комнаты'])
data.insert(7, 'Номер №', data['Комн.№'])
data.insert(8, 'Кол-во номеров', data['Комнат'])
data.insert(11, 'Суток', (data['Выезд'] - data['Заезд']).dt.days)
data.insert(16, 'Тариф', data['Тариф_y'])
data.insert(17, 'Доход', (data['Тариф']*data['Суток']))

Из-за некорректных дат рождения, алгоритм падает.
Раскоментить строки, если это произошло, при необходимости повторить блок:

In [9]:
nat = pd.to_datetime('ff',errors='coerce')

In [10]:
#data['Д. рожд'] = data['Д. рожд'].replace(data['Д. рожд'].min(),nat)
#data['Д. рожд'] = data['Д. рожд'].replace(data['Д. рожд'].max(),nat)

#Потом нужно загнать это в цикл и try..except

In [11]:
data.insert(20, 'Возраст', np.floor(((data['Заезд'] - data['Д. рожд']).dt.days+1)/365))

In [12]:
data = data.drop(labels=['Комн.№','Комнат','Тип комнаты','Тариф_y'], axis=1)


Даже в отмененных бронях сохраняется информация о дате заезда. Если этой даты нет, то бронь носила технический характер, её можем дропнуть

In [13]:
data = data.dropna(subset='Заезд')

### Текстовая обработка

#### Обработка телефонов и почты

Периодическая ошибка: почта записывается в графу телефона, возникает при ручном вводе и через некорректные API от турагентств

**Порядок ранее применявшейся ручной обработки номеров:**

1.	Если есть email в столбце телефонов переместить значение в столбец email, маркер email - знак @
2.	Столбец телефонов Убрать: пробелы, +, -, (,)
3.	Преобразовать текст в числа в телефонах
4.	Убрать текстовые примечания из номера
5.	Убрать слишком короткие и слишком длинные номера (ошибка ручного ввода)
6.	Добавить 7 к номерам формата 9ххххххххх
7.	Заменить 8 на 7 в номерах формата 89ххххххххх


**Поставим email на место**

In [14]:
data.loc[((data['Tel'].str.contains('@', regex=True)).fillna(False)) & (data['Email'].isna()),'Email'] = data[((data['Tel'].str.contains('@', regex=True)).fillna(False)) & (data['Email'].isna())]['Tel']

**Обогатим данные:**

Попытаемся восстановить телефоны из адресов email, т.к. иногда адрес почты совпадает с телефоном.

In [15]:
data.loc[~(data['Email'].isna()) & (data['Tel'].isna()),'Tel'] = data.loc[~(data['Email'].isna()) & (data['Tel'].isna()),'Email']

In [16]:
#data[((data['Tel'].str.contains('@', regex=True)).fillna(False))]

Email сохранены максимально. Далее можем не бояться убрать все буквенные значения из Телефонов.
Напишем функцию обработки 1 телефона, она будет возвращать телефон в формате 7ХХХХХХХХХХ или NAN, если не удалось перевести телефон в такой формат.

Основные ошибки ввода номера:
- Разный формат +7-ХХХ-ХХХ-ХХ-ХХ 8-ХХХ-ХХХ-ХХ-ХХ
- дефисы, пробелы, скобки, плюсики и другие разделители в телефоне
- комментарии которые прописывают в номер: Даша (жена)

In [17]:
a = '+8(3513)29-80 91 Фёдор, звонить вечером'

In [18]:
def true_phone(bad):
    """ Вычленяет только цифры из всех символов в телефонном номере, возвращает номер в формате 7ХХХХХХХХХХ, если значение нельзя привести к данному формату, возвращает NAN """
    try:
        bad = str(bad)
        numlst=["0","1","2","3","4","5","6","7","8","9"]
        countfinish=0
        good=""
        for charr in bad:
            for num in numlst:
                if num==charr:
                    good=good+str(num)
        if 70000000000<int(good)<79999999999: return int(good)
        elif 1000000000<int(good)<9999999999: return (int(good) + 70000000000)
        elif 80000000000<int(good)<89999999999: return (int(good) - 10000000000)    
        else: return np.nan
        
    except:
        return np.nan

In [19]:
true_phone(a)

73513298091

In [20]:
data.insert(25, 'Телефон', np.nan)

In [21]:
data['Телефон'] = data['Tel'].apply(true_phone)

In [22]:
data['Телефон'] = data['Телефон'].astype('Int64')
data['Id проф'] = data['Id проф'].astype('Int64')

In [23]:
data.loc[~(data['Tel'].isna()) & (data['Телефон'].isna())]

,ID,Отель,Res ID,Дата созд,Окно бронир,Код тарифа,Категория номера,Номер №,Кол-во номеров,Заезд,Выезд,Суток,Взр,Дет,Статус,Код источника,Тариф,Доход,Id проф,Д. рожд,Возраст,Город,ФИО,Email,Tel,Телефон,Дата созд. проф.,Накопл. ночей,Накопл. заездов,Группа,Компания,Агент,Коды рынка,Менеджер


При тестовой прогонке на детесете за 6 лет мы потеряли всего 0,4% контактов по следующим причинам:
- указана почта вместо номера
- указанный номер короче телефонного
- указанный номер длиннее телефонного

#### Обработка ФИО

Убрать ошибки из имен: «пробел + запятая» заменить на «запятая + пробел», «пробел + пробел» на «пробел».

Прочие опечатки убирать пока нецелесообразно, т.к. ключевые идентификаторы профайла его ID или телефон

In [24]:
#data['ФИО'] = data['ФИО'].capitalize()

In [25]:
data['ФИО'] = data['ФИО'].replace(' ,',', ')
data['ФИО'] = data['ФИО'].replace('  ',' ') 

#### Обработка городов

Города вводятся вручную, все ошибки и опечатки администратора сохраняются в базе.
Города важны для аналитики, т.к. в разбивке по региону вычисляются ключевые метрики.

1. Приводим города к единому регистру
2. Удаляем мусор "город" "село" и т.д.
3. Исправляем частые ошибки по словарю
4. Возвращаем первую заглавную букву

#### Подфункция замены по словарю

In [26]:
def rep_dict (string, dict):
    """ Функция устраняет частые опечатки из заготовленного словаря городов, подфункция для обработки городов """
    for incorrect, correct in dict.items():
        string = string.replace(incorrect, correct)
    return string

In [27]:
city_dict = {'челяюинск':'челябинск','члб':'челябинск','трех горный':'трехгорный',
    'ак сахарова':'челябинск','асбет':'асбест','альменьево':'альменево','чеябинск':'челябинск','ектеринбург':'екатеринбург',
    'екаеринбург':'екатеринбург','екаберинбург':'екатеринбург','екаринбург':'екатеринбург',
    'екаиеринбург':'екатеринбург','екб':'екатеринбург','екетеринбург':'екатеринбург','еккатеринбург':'екатеринбург',             
    'магнигороск':'магнитогорск','магниогорск':'магнитогорск','магнитакорск':'магнитогорск','магнитвагорск':'магнитогорск',
    'магнитогрск':'магнитогорск','магниторгск':'магнитогорск','магниторогск':'магнитогорск','москве':'москва','перми':'пермь',
    'альментьевск':'альметьевск','асберт':'асбест','асбеста':'асбест','березовкий':'березовский','березовской':'березовский',
    'березовском':'березовский','березоский':'березовский','седельникова':'седельниково','вавмловец':'вавиловец','вавилонец':'вавиловец',
    'вавмловец':'вавиловец','верхня ':'верхняя ','глазова':'глазов','долгодеоевенское':'долгодеревенское','долгодеревенская':'долгодеревенское',
    'долгопродный':'долгопрудный','ззлатоуст':'златоуст','злаоуст':'златоуст','зластоуст':'златоуст','златуст':'златоуст',
    'каменск уральский':'каменск-уральский','каменск уральскии':'каменск-уральский','каменк-уральский':'каменск-уральский','крменкуль':'кременкуль',
    'круган':'курган','куган':'курган','кургана':'курган','магнитагорск':'магнитогорск','магнитогорска':'магнитогорск','миассс':'миасс',
    'москав':'москва','моска':'москва','перьм':'пермь','стерлитаак':'стерлитамак','стерлитамака':'стерлитамак','стерлитмак':'стерлитамак',
    'стерлитомак':'стерлитамак','стермитамак':'стерлитамак','сторлытамак':'стерлитамак','тольяти':'тольятти','усть катав':'усть-катав','уфы':'уфа',
    'ханты мансийск':'ханты-мансийск','хантымансийск':'ханты-мансийск','ялутовск':'ялуторовск','пашма':'пышма','паша':'пышма',
    'ростов на дону':'ростов-на-дону','санк-петербург':'санкт-петербург','сакт-петербург':'санкт-петербург','сакнт-петербург':'санкт-петербург',
    'санкт петербург':'санкт-петербург','санкт-петергбург':'санкт-петербург','санкт-питербург':'санкт-петербург','Свердловск':'екатеринбург',         
    'санкт-перебург':'санкт-петербург','санкт-перербург':'санкт-петербург','санкт-петепбург':'санкт-петербург','сарапуль':'сарапул',        
    'стерлимак':'стерлитамак','среднеуарльск':'среднеуральск','среднеурвльск':'среднеуральск','среднеурльск':'среднеуральск','уреньгой':'уренгой',
    'увелььский':'увельский','каменс-уральск':'каменск-уральск','вотскинск':'воткинск','ханты-мансйиск':'ханты-мансийск','ханты-манскийск':'ханты-мансийск',
    'ханта манси':'ханты-мансийск','чнежинск':'снежинск','артеовский':'артемовский','брюзань':'юрюзань','еременкуль':'кременкуль','западний':'западный','заподный':'западный',
    'н.тагил':'нижний тагил','нефтеюганске':'нефтеюганск','харлугии':'харлуши','кременкульодной':'кременкуль','свердловская обл., туринский р-он, туринск':'туринск','туринскй р-н':'туринск'         
    }

In [28]:
def good_city(city):
    """ Функция обработки городов: убирает тип населенного пункта, исправляет опечатки, типографские ошибки, вызывает исправление опечаток по словарю """
    city = str(city)
    city = city.lower()
    city = city.replace('ё','е')
    city = city.replace(' - ','-')
    city = city.replace('- ','-')
    city = city.replace(' -','-')

    city = city.replace('город. ','')
    city = city.replace('деревня ','')
    city = city.replace('гор. ','')
    city = city.replace('пос. ','')
    city = city.replace('г. ','')  
    city = city.replace('с. ','')
    city = city.replace('р. ','')
    city = city.replace('дер. ','') 
    city = city.replace('д. ','') 
    city = city.replace('п. ','') 
    city = city.replace('пгт. ','')
    city = city.replace('ст. ','')
    city = city.replace('мкр. ','')

    city = city.replace('город.','')
    city = city.replace('гор.','')
    city = city.replace('пос.','')
    city = city.replace('г.','') 
    city = city.replace('с.','')
    city = city.replace('р.','')
    city = city.replace('дер.','') 
    city = city.replace('д.','') 
    city = city.replace('п.','') 
    city = city.replace('пгт.','')
    city = city.replace('ст.','')
    city = city.replace('р.п.','')
    city = city.replace('мкр.','')
    
    city = city.replace('город ','')
    city = city.replace('поселок ','')
    city = city.replace('гор ','')
    city = city.replace('пос ','')
    city = city.replace('г ','') 
    city = city.replace('с ','') 
    city = city.replace('дер ','') 
    city = city.replace('д ','') 
    city = city.replace('п ','') 
    city = city.replace('пгт ','')
    city = city.replace('ст ','')
    city = city.replace('село ','')
    city = city.replace('мкр ','')
    
    city = city.replace(' поселок','')
    city = city.replace(' село','')
    city = city.replace(' мкр.','')
    city = city.replace(' мкр','')
    city = city.replace(' дер.','')
    city = city.replace(' дер','')
    city = city.replace(' пгт.','')
    city = city.replace(' пгт','')
    
    city = rep_dict (city, city_dict)
    
    city = city.replace(' ,',', ')
    city = city.replace('  ',' ')
    if city[0] == '.': city = city.replace('.','')
    if city[0] == ' ': city = city.replace(' ','')
    if (city[-1] == 'г') and (city[-2] == ' '): city = city.replace(' г','')
    if (city[-1] == 'п') and (city[-2] == ' '): city = city.replace(' п','')
    if (city[-1] == 'д') and (city[-2] == ' '): city = city.replace(' д','')    
    if (city[-1] == 'с') and (city[-2] == ' '): city = city.replace(' с','')
    if (city[-1] == 'с') and (city[-2] == 'о') and (city[-3] == 'п') and (city[-4] == ' '): city = city.replace(' пос','')
    if (city[-1] == ' '): city = city.replace(' ','')
    if (city[0] == 'е') and (city[1] == 'к') and (city[2] == 'а') and (city[3] == 'т'): city = 'екатеринбург' 
    if (city[0] == 'ч') and (city[1] == 'е') and (city[2] == 'л') and (city[3] == 'б'): city = 'челябинск'    
    if (city[0] == 'ч') and (city[1] == 'е') and (city[2] == 'л') and (city[3] == 'я'): city = 'челябинск' 
    if (city[0] == 'ч') and (city[-1] == 'к') and (city[-2] == 'с') and (city[-3] == 'н') and (city[-4] == 'и') and (city[-5] == 'б'): city = 'челябинск'
    if city[0:8] == 'западный': city = 'челябинск' 
    if (city == 'каменск-уральск') or (city == 'каменск уральск'): city = 'каменск-уральский'
    if (city == 'златоус'): city = 'златоуст'
    if (city[0:3] == 'миас') and (city[0:5] == 'миасск'): city = 'миасс'
    city = city.capitalize()
    if city == 'Nan': return np.nan
    else: return city

In [29]:
data['Город'] = data['Город'].apply(good_city)

#### Определяем регионы

Определяем регион по городу. 

Готовая база из 1200 крупнейших городов России дополнена данными о малых населенных пунктах Челябинской и Свердловской области.

In [30]:
cities = pd.read_excel ('cities.xlsx')

In [31]:
cities = cities.drop(labels=['№','Население'], axis=1)
cities = cities.rename(columns={'Регион':'Регион_'})
cities

,Город,Регион_,Федеральный округ
0,Абаза,Хакасия,Сибирский
1,Абакан,Хакасия,Сибирский
2,Абдулино,Оренбургская обл.,Приволжский
3,Абинск,Краснодарский край,Южный
4,Агидель,Башкортостан,Приволжский
...,...,...,...
1294,Усть-качка,Пермский край,Приволжский
1295,Богандинский,Тюменская обл.,Уральский
1296,Новое девяткино,Санкт-Петербург и ЛО,Северо-Западный
1297,Мармылева,Башкортостан,Приволжский


In [32]:
data = data.merge(cities, how='left', on='Город')

In [33]:
data.insert(22, 'Регион', data['Регион_'])
data.insert(23, 'Фед.Окр', data['Федеральный округ'])
data = data.drop(labels=['Регион_'], axis=1)
data = data.drop(labels=['Федеральный округ'], axis=1)


### Обогащение данных

Привлечение гостей из новых регионов - одна из стратегических задач маркетинга.

Необходимо собрать максимально подробную информацию о регионе. Обогатить данные можно по номеру телефона.

Обогащение данных о регионе по номеру телефона
+ Не забыть про казахстан +77

**На базе открытых источников сформирован датасет о принадлежности номера к региону**

Датасет имеет структуру: 
- начало интервала номеров
- конец интервала номеров
- регион телефона
- федеральный округ

Методика не учитывает переход от оператора к оператору и переезды между регионами, однако это лучшее из возможных решений. Опять же даже данные о прописке не гарантируют совпадение региона проживания с пропиской.

In [34]:
phones = pd.read_excel ('phones.xlsx')
phones

,От,До,Регион,Федеральный округ
0,70000000000,76999999999,NaN,NaN
1,77000000000,77999999999,КАЗАХСТАН,NaN
2,78000000000,73429999999,NaN,NaN
3,73430000000,73439999999,Свердловская обл.,Уральский
4,73440000000,73509999999,NaN,NaN
...,...,...,...,...
7549,79964600000,79964629999,Амурская обл.,Дальневосточный
7550,79991650000,79991669999,Амурская обл.,Дальневосточный
7551,79992515000,79992519999,Амурская обл.,Дальневосточный
7552,79992547000,79992548999,Амурская обл.,Дальневосточный


In [35]:
phones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7554 entries, 0 to 7553
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   От                 7554 non-null   int64 
 1   До                 7554 non-null   int64 
 2   Регион             7548 non-null   object
 3   Федеральный округ  7547 non-null   object
dtypes: int64(2), object(2)
memory usage: 236.2+ KB


#### Подфункция обогащения регионов

In [36]:
#p = 79226963650

In [37]:
# Рабочий но очень долгий вариант. На базе за 6 лет работал около 40 минут. На ежемесячной выгрузке справляется за 5-7 минут.
# Если нужно будет масштабировать, можно подумать над индексированным поиском.

def tel_region (row):
    """ Функция проверяет запись датасета, если есть телефон, но не указан регион, пытается найти телефон в диапазонах справочника и дозаполнить регион """
    if pd.isna(row['Регион']) and not pd.isna(row['Телефон']):
        for index, code in phones.iterrows():
            if (row['Телефон'] >= code['От']) and (row['Телефон'] <= code['До']): 
                row['Регион'] = code['Регион']
                row['Фед.Окр'] = code['Федеральный округ']
                break
    return row

In [38]:
data = data.apply(tel_region, axis=1)

#### На будущее

Можно дообогатить данные по скрытым дубликатам профайлов клиента, когда на 1 гостя было заведено более 1 профайла.

Но лучше обогатить данные слиянием нескольких баз отелей (проверить согласие на обработку ПД)
1. Слияние баз
2. Новый GlobalID по номеру телефона
3. Если в рамках GlobalID есть совпадения по ФИО и пропуски по дате рождения городу - заполнить пропуски.
4. Разделение баз
5. Анализ отеля

### Правка датасета

#### Удаление аномалий по возрасту

Возраст тоже проставляется вручную. Иногда администратор может ошибочно проставить вместо даты рождения дату заселения или дату выдачи паспорта:

С 14 лет по законам РФ должно быть наториальное согласие родителей для заселение в отель. - ОК

С 18 лет граждане РФ могут заселяться в отель самостоятельно.

Гости старше 90 лет тоже выглядят подозрительно.

Удалить возраст и дату рождения при возрасте меньше 14 и больше 90. Т.к. нам нужно очистить и исходную дату рождения и вычисленный возраст, напишем построчную функцию.

In [39]:
def ageclear(row):
    """ Зачищает аномальный возраст и дату рождения в строке """
    if (row['Возраст'] < 14) or (row['Возраст'] > 90): 
        row['Д. рожд'] = nat
        row['Возраст'] = np.nan
    return row

In [40]:
data = data.apply(ageclear, axis=1)

#### Добавляем идентификаторы 
Для удобного последующего сегментирования в BI системе:
- бронь с детьми
- бронь гостя с более чем 1 бронью (условно повторный гость)
- бронь с сайта

In [41]:
data['С детьми'] = 0

In [42]:
data.loc[data['Дет'] > 0,'С детьми'] = 1

In [43]:
data['Повторный'] = 0

In [44]:
data.loc[data['Накопл. заездов'] > 1,'Повторный'] = 1

In [45]:
data['Из TL'] = 0

In [46]:
data.loc[(data['Код тарифа'].str.contains('TL', case=False, regex=True)).fillna(False),'Из TL'] = 1
data.loc[(data['Агент'] == 'Travelline').fillna(False),'Из TL'] = 1
data.loc[(data['Код источника'] == 'TL').fillna(False),'Из TL'] = 1    

#### Переорпеделяем сегменты гостей

Примечание: не менять последовательность блоков, она выверена логикой приоритетов сегментации.

In [47]:
data['СЕГМЕНТ'] = 'NONE'

In [48]:
data.loc[(data['Из TL'] == 1),'СЕГМЕНТ'] = 'IND'
data.loc[(data['Код тарифа'].str.contains('код_1', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'OWN'
data.loc[(data['Код тарифа'].str.contains('код_2', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'OWN'
data.loc[(data['Код тарифа'].str.contains('код_3', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'SPORT'
data.loc[(data['Код тарифа'].str.contains('код_4', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'SPORT'
data.loc[(data['Код тарифа'].str.contains('код_5', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'SPORT'
data.loc[(data['Код тарифа'].str.contains('код_6', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'FEST'
data.loc[(data['Код тарифа'].str.contains('код_7', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Код тарифа'].str.contains('код_8', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Код тарифа'].str.contains('код_9', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'PROJ'
data.loc[(data['Код тарифа'].str.contains('код_10', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'PROJ'
data.loc[(data['Код тарифа'].str.contains('код_11', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'PROJ'
data.loc[(data['Код тарифа'].str.contains('код_12', case=False, regex=True)).fillna(False),'СЕГМЕНТ'] = 'FEST'
data.loc[(data['Из TL'] == 1),'СЕГМЕНТ'] = 'IND'

In [49]:
data.loc[~(data['Агент'].isna()) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'IND'
data.loc[(data['Компания'].str.contains('Партнер_1', case=False, regex=True)).fillna(False) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Компания'].str.contains('Партнер_2', case=False, regex=True)).fillna(False) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Компания'].str.contains('Партнер_3', case=False, regex=True)).fillna(False) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Компания'].str.contains('Партнер_4', case=False, regex=True)).fillna(False) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'PROJ'

In [50]:
data.loc[(data['Менеджер'].str.contains('Фамилия менеджера 1', case=False, regex=True)).fillna(False) & ~(data['Группа'].isna()),'СЕГМЕНТ'] = 'SPORT'
data.loc[(data['Менеджер'].str.contains('Фамилия менеджера 2', case=False, regex=True)).fillna(False) & ~(data['Группа'].isna()),'СЕГМЕНТ'] = 'SPORT'
data.loc[(data['Менеджер'].str.contains('Фамилия менеджера 3', case=False, regex=True)).fillna(False) & ~(data['Группа'].isna()),'СЕГМЕНТ'] = 'CORP'
data.loc[(data['Менеджер'].str.contains('Фамилия менеджера 4', case=False, regex=True)).fillna(False) & ~(data['Группа'].isna()),'СЕГМЕНТ'] = 'CORP'
data.loc[(data['Менеджер'].str.contains('Фамилия менеджера 5', case=False, regex=True)).fillna(False) & ~(data['Группа'].isna()),'СЕГМЕНТ'] = 'PROJ'
data.loc[(data['Код тарифа'] == 'код_13') & (data['СЕГМЕНТ'] == 'NONE') & (data['Тариф'] == '0'),'СЕГМЕНТ'] = 'SPEC'
data.loc[(data['Коды рынка'] == 'код_14') & (data['Тариф'] == '0'),'СЕГМЕНТ'] = 'PROJ'
data.loc[~(data['Группа'].isna()) & (data['СЕГМЕНТ'] == 'NONE'),'СЕГМЕНТ'] = 'CORP'

In [51]:
data['СЕГМЕНТ'] = data['СЕГМЕНТ'].replace('NONE','IND')

#### Разбиаение по корпусам для некоторых отелей

In [52]:
if hotel == 'RR':
    data.loc[data['Категория номера'].isin(['код_2','код_3','код_4','код_5','код_6','код_7']),'Отель'] = 'AA'
    data.loc[data['Категория номера'].isin(['код_8','код_9']),'Отель'] = 'PP'

In [53]:
if hotel == 'BB':
    data.loc[data['Категория номера'].isin(['код_10','код_11','код_12','код_13','код_14','код_15','код_16','код_17','код_18']),'Отель'] = 'BB'
    data.loc[data['Категория номера'].isin(['код_19','код_20','код_21','код_22','код_23','код_24']),'Отель'] = 'TT'
    data.loc[data['Категория номера'].isin(['код_25','код_26','код_27','код_28']),'Отель'] = 'CC'
    data.loc[data['Категория номера'].isin(['код_29','код_30']),'Отель'] = 'DD'

## Вывод полного датасета

In [54]:
data.to_excel('GG_processed_data.xlsx', index=False)

## Вывод датасета для Бизнес-Аналитики без чувствительных данных

Также уберем данные по аренде не номеров.

In [55]:
if hotel == 'BB':
    data = data[data['Код тарифа'] != 'BAN']
    data = data[data['Категория номера'] != 'BAN']

In [56]:
data = data.drop(labels=['ФИО','Email','Tel','Телефон'], axis=1)

In [57]:
data.to_excel('GG_data_for_bi.xlsx', index=False)